# Tape Analysis

In [12]:
import pandas as pd
import datetime


In [8]:
df = pd.read_excel('input/cvx.xlsx', index_col = "Date") #import data from excel file

In [22]:
df.drop(columns=['Bid','Ask','Sizes'],inplace=True) # delete columns


In [27]:
df.dropna(how='any',inplace=True)

In [31]:
df.sort_index(inplace = True)

In [32]:
with pd.ExcelWriter("output/output.xlsx") as writer:
   
    df.to_excel(writer, sheet_name="output")
    